so my plan is that i would have 2 different iterations of the dataset: one iteration would be the raw dataset. The second iteration would have the resultant vector from every consecutive frame that would provide us with the vector values of the movement of the observed landmark. Now both of the datasets would be reshapes to better resemble an image even if they are far from an recognisable image. If the resizing is not feasable, we would transform the "images" into a uniform shape so that it could pass the model. 

After that is done, i would create a model with 2 convolutional branches that woudl recognize teh different characteristics of the "images", and whatever features it was able to extract from the "images" woould be utilized in the final prediction process as the two braches merge into each other(note that the vanilla version of the dataset brach would be be independant, but the other branch would be heavily influenced by the vanilla branch. This is done so that landmark movement vector has a reference point in space. 

The outputs of the model, both for training and validation, would also be transformed into image-like state so as to simplify the prediction process(note that since not all of the clips do not have the same frames, there woudl be a few null values, and to take care of those values we would just replace those values with 0. This is done because the data we are using are spatial data, and any misrepresentation could drastically affect the prediction of the model.

Once we have the prediction, we woudld flatten out the data and then create another model, but this time it would be different, as i would be making a transfomer neural network. At the point of writting this manifesto, i have very little knowledge about transformer neural networks, so once i have gathered sufficient data regarding these model- I will be optimistic and vague.

In [24]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils import data
import torchvision
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import smart_resize
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import tokenizer_from_json,Tokenizer
from tqdm import notebook, tqdm
from tqdm.notebook import tnrange
from typing import List, Tuple, Dict
from collections import defaultdict
from pathlib import Path
import json
import logging
import gc
import os

In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Available devices: {device}")

Available devices: mps


In [3]:
with open("/Users/arvinprince/pytorch-files/Google ASL/data/character_to_prediction_index.json") as json_file:
    CHAR2ORD = json.load(json_file)

In [4]:
train_csv = pd.read_csv("/Users/arvinprince/pytorch-files/Google ASL/data/train.csv")

In [5]:
df = pd.read_parquet("/Users/arvinprince/pytorch-files/Google ASL/data/296317215.parquet").astype(np.float16)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165396 entries, 1494438637 to 1526914806
Columns: 1630 entries, frame to z_right_hand_20
dtypes: float16(1630)
memory usage: 515.5 MB


In [7]:
print(f"Shape of the dataset: {df.shape}")
df

Shape of the dataset: (165396, 1630)


,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
sequence_id,,,,,,,,,,,,,,,,,,,,,
1494438637,0.0,0.558105,0.550781,0.554199,0.543945,0.551270,0.555176,0.566406,0.455322,0.572266,...,-0.124939,-0.113708,-0.103394,-0.118713,-0.091064,-0.071289,-0.095154,-0.091675,-0.064819,-0.046692
1494438637,1.0,0.559082,0.550781,0.554199,0.543457,0.551270,0.554688,0.565430,0.455566,0.571289,...,-0.101074,-0.092773,-0.083496,-0.102844,-0.077515,-0.058441,-0.076904,-0.079407,-0.056061,-0.039825
1494438637,2.0,0.555176,0.551270,0.554688,0.543457,0.551758,0.555176,0.565430,0.454346,0.570801,...,-0.139404,-0.124512,-0.095825,-0.131958,-0.104431,-0.075500,-0.096985,-0.116089,-0.090149,-0.064758
1494438637,3.0,0.557129,0.550293,0.553711,0.542969,0.550781,0.554199,0.564453,0.454834,0.570312,...,-0.187744,-0.179443,-0.133179,-0.173218,-0.143188,-0.115540,-0.135498,-0.150513,-0.123901,-0.102112
1494438637,4.0,0.557617,0.551270,0.554688,0.543945,0.552246,0.555664,0.565918,0.454590,0.571777,...,-0.167480,-0.152832,-0.127319,-0.171875,-0.133301,-0.097473,-0.145142,-0.160400,-0.122375,-0.090515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526914806,21.0,0.625000,0.602051,0.613281,0.584473,0.598633,0.598633,0.602051,0.485596,0.598633,...,-0.346924,-0.397217,-0.216919,-0.324951,-0.403564,-0.456055,-0.235718,-0.330322,-0.371826,-0.400146
1526914806,22.0,0.625488,0.600586,0.612305,0.583496,0.597168,0.597656,0.600586,0.484619,0.597656,...,-0.189209,-0.222778,-0.099060,-0.173584,-0.230835,-0.266113,-0.110291,-0.190186,-0.219727,-0.233276
1526914806,23.0,0.623047,0.602051,0.613770,0.584473,0.598633,0.598633,0.601562,0.485352,0.598145,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
par_path = Path("/Users/arvinprince/pytorch-files/Google ASL/data/296317215.parquet")
phrases = train_csv.groupby("file_id").get_group(int(par_path.stem)
                                      ).sort_values(by="sequence_id").phrase.tolist()
phrases[:5]

['gladys guzman',
 '8780 old 13th',
 '+62-879-37-82',
 'www.china-ycyb.com/arduinoprof',
 '7642 south beulah road']

In [101]:
def character_indexer(string:str):
    return np.array(list(map(lambda x: CHAR2ORD[x] +2, list(string))), dtype=np.uint16)

In [102]:
tokenizer = tf.keras.layers.TextVectorization(max_tokens=61,
                                              split="character",
                                              standardize="lower",
                                              output_mode="int",
#                                               pad_to_max_tokens=True,
                                              vocabulary=tuple(CHAR2ORD.keys()),
                                              )

In [105]:
# tokenizer(phrases[0]) == character_indexer(string=phrases[0])
tokenizer(phrases)[0]

<tf.Tensor: shape=(31,), dtype=int64, numpy=
array([40, 45, 34, 37, 58, 52,  2, 40, 54, 59, 46, 34, 47,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])>

In [104]:
CHAR2ORD

{' ': 0,
 '!': 1,
 '#': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 '*': 9,
 '+': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '=': 27,
 '?': 28,
 '@': 29,
 '[': 30,
 '_': 31,
 'a': 32,
 'b': 33,
 'c': 34,
 'd': 35,
 'e': 36,
 'f': 37,
 'g': 38,
 'h': 39,
 'i': 40,
 'j': 41,
 'k': 42,
 'l': 43,
 'm': 44,
 'n': 45,
 'o': 46,
 'p': 47,
 'q': 48,
 'r': 49,
 's': 50,
 't': 51,
 'u': 52,
 'v': 53,
 'w': 54,
 'x': 55,
 'y': 56,
 'z': 57,
 '~': 58}

In [ ]:
"pizza" "burger" "steak"
0.35 0.33 .31

In [81]:
tokenizer.get_vocabulary()
# len(tuple(CHAR2ORD.keys()))

['',
 '[UNK]',
 ' ',
 '!',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '?',
 '@',
 '[',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '~']

In [11]:
# n = df.fillna(0).groupby("sequence_id").get_group(1526914806
#                                                  ).diff(axis=0).dropna().values[:,1:546]

In [12]:
# n.flatten()

In [27]:
class ASLFR_Dataset(data.Dataset):
    def __init__(self,
                 landmark_p: Path,
                 landmark_df: pd.DataFrame,
                 phrase: pd.DataFrame):
        self.l = landmark_p
        self.df = landmark_df.astype(np.float32).reset_index()
        
        self.df.fillna(0, inplace=True)
        self.df.drop([col for col in self.df.columns if "z" in col], axis = 1, inplace=True)
        self.df.drop([col for col in self.df.columns if "pose" in col and int(col[7:]) >= 25], 
                axis = 1, inplace=True)
        self.df.drop("frame", axis = 1, inplace=True)
        
        self.p = train_csv.groupby("file_id").get_group(int(self.l.stem)
                                                       ).sort_values(by="sequence_id").phrase.values
    
#     def load_dataset(self):
#         return pd.read_parquet(self.l).astype(np.float32).reset_index()
    
#     def filtering_coordinates(self,
#                             df:pd.DataFrame):
#         df.fillna(0, inplace=True)
#         df.drop([col for col in df.columns if "z" in col], axis = 1, inplace=True)
#         df.drop([col for col in df.columns if "pose" in col and int(col[7:]) >= 25], axis = 1, inplace=True)
# #         df.drop([y for y in df.columns if "y" in y], inplace= True, axis = 1)
#         df.drop(["frame"], axis=1, inplace=True)
#         return df
    
    def character_indexer(self,string:str):
        return np.array(list(map(lambda x: CHAR2ORD[x], list(string))), dtype=np.uint32)
    
    def __len__(self):
        return len(self.p)
    
    def __getitem__(self, 
                    index:int):
        s_id = self.df.sequence_id[index]
        phrase = self.p[index]
#         tensor = smart_resize(
        x_val = self.df.groupby("sequence_id"
                               ).get_group(s_id).diff().dropna().values[:,1:536]
        y_val = self.df.groupby("sequence_id"
                               ).get_group(s_id).diff().dropna().values[:,536:]
#         tensor = np.dstack((self.df.groupby("sequence_id").get_group(s_id).values[:,1:536],
#                        self.df.groupby("sequence_id").get_group(s_id).values[:,536:])),
        tensor = (np.sqrt(np.square(x_val)+np.square(y_val)))
#             (256,256)
#         )
        return (tensor.flatten(), self.character_indexer(phrase))

In [28]:
train_ds = ASLFR_Dataset(landmark_p=Path("/Users/arvinprince/pytorch-files/"
                              "Google ASL/data/296317215.parquet"),
              landmark_df=df,
              phrase=train_csv
             )

In [29]:
np.save("X.npy", np.array(train_ds))

/var/folders/38/jtldm9cn7qjfn516_22dcgcm0000gn/T/ipykernel_33153/2328165485.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.save("X.npy", np.array(train_ds))


here are the things to be done if you forget <br>
create a training dataset by taking out the coords and then mathcing them with the phrases and then passing them through a model

In [90]:
X, y = np.load("X.npy", allow_pickle=True).transpose()

In [185]:
sequence = y[0]
input_sequence=[]
for i in range(1, len(sequence)):
    input_sequence.append(sequence[:i+1])

In [206]:
input_sequence

[array([38, 43], dtype=uint32),
 array([38, 43, 32], dtype=uint32),
 array([38, 43, 32, 35], dtype=uint32),
 array([38, 43, 32, 35, 56], dtype=uint32),
 array([38, 43, 32, 35, 56, 50], dtype=uint32),
 array([38, 43, 32, 35, 56, 50,  0], dtype=uint32),
 array([38, 43, 32, 35, 56, 50,  0, 38], dtype=uint32),
 array([38, 43, 32, 35, 56, 50,  0, 38, 52], dtype=uint32),
 array([38, 43, 32, 35, 56, 50,  0, 38, 52, 57], dtype=uint32),
 array([38, 43, 32, 35, 56, 50,  0, 38, 52, 57, 44], dtype=uint32),
 array([38, 43, 32, 35, 56, 50,  0, 38, 52, 57, 44, 32], dtype=uint32),
 array([38, 43, 32, 35, 56, 50,  0, 38, 52, 57, 44, 32, 45], dtype=uint32)]

In [ ]:
# X= tf.ragged.constant(X.tolist())

In [ ]:
y = 

In [199]:
X[0].shape

(1,)

In [91]:
model = keras.Sequential()
# model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(64, (5,5)))
model.add(keras.layers.MaxPool2D((2,2)))
model.add(keras.layers.Conv2D(64, (3,3)))
model.add(keras.layers.MaxPool2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation="relu"))
model.add(keras.layers.Dense(64, activation="tanh"))
# model.add(keras.layers.Embedding(64,240))
# model.add(keras.layers.Bidirectional(keras.layers.LSTM(150)))
model.add(keras.layers.Dense(31, activation="softmax"))

In [92]:
model.compile(loss="categorical_crossentropy",
             optimizer= keras.optimizers.legacy.Adam(learning_rate=0.01), 
             metrics=["accuracy"])

In [93]:
model.fit(X, y, epochs=3)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [176]:
l = model.predict(X[0].reshape((1,256,256,2)))

1/1 [==============================] - 0s 38ms/step


array([[1.3485312e-26, 7.9447412e-12, 8.3316898e-08, 3.0691541e-07,
        6.4937849e-06, 2.4789362e-05, 3.8392318e-05, 3.5051026e-04,
        5.9117848e-04, 1.6003705e-03, 1.5310565e-03, 7.9629786e-04,
        1.3677583e-02, 3.4777613e-03, 3.9988495e-03, 6.1395294e-03,
        4.8061758e-03, 5.0305882e-03, 2.4779378e-02, 3.8109012e-02,
        5.1720563e-02, 5.7593562e-02, 8.3091762e-03, 1.5193112e-01,
        9.8334633e-02, 7.5005894e-03, 2.3799855e-01, 9.8285884e-02,
        9.1578579e-03, 3.8220968e-02, 1.3598874e-01]], dtype=float32)